<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga librerías</span>

In [1]:
import ultralytics
import sklearn
from ultralytics import YOLO
from ultralytics import settings
import os
import torch
import subprocess
import gdown
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga rutas</span>

In [2]:

dir_main= os.getcwd() # Obtener ubicación actual
main_folder = os.path.abspath(os.path.join(dir_main, "../../../")) # Carpeta main del proyecto

dir_train= os.path.abspath(main_folder) + "\\dist\\lote2_2" # ruta archivos de entrenamiento y validación json

dir_training_experiments= os.path.abspath(main_folder) + "\\src\\srcv2\\segmentacion\\segmentacion" # ruta modelo entrenado
def get_last_training(dir_train):
    experiments = os.listdir(dir_train)
    order_of_experiments = sorted(item for item in experiments if os.path.isdir(os.path.join(dir_train, item)))
    if order_of_experiments:
        return f'{dir_train}\\{order_of_experiments[-1]}\\weights\\best.pt'
    else:
        return None
model_train= get_last_training(dir_training_experiments) # Ultimo modelo entrenado

dir_prueba= os.path.abspath(main_folder) + "\\src\\assets"  # Carpeta de imagenes pruebas

In [ ]:
# Modificación a la configuración del dataset 
dir_train_converted = os.path.abspath(main_folder) + "\\dist\\data\\dataset.yaml" 
update_dataset= os.path.abspath(os.path.join(dir_train_converted, "../"))
with open(dir_train_converted, 'w') as new_dataset:
    new_dataset.write(
        f'train: {update_dataset}\\train\n'
        f'val: {update_dataset}\\val\n'
        'test: \n'
        'nc: 2 \n'
        'names: ["rayon", "abolladura"]\n'
    )

In [ ]:
# Modificación de las rutas de las carpetas predeterminadas de ultralytics YOLO
main= os.path.abspath(main_folder) + '\\dist'
def set_config_ultralytics(main):
    settings.update({'datasets_dir': f'{main}\\data', 'weights_dir': f'{main}', 'runs_dir': f'{main}\\runs'})
set_config_ultralytics(main)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de herramientas de etiquetado</span>

In [ ]:
# Función para instalar los paquetes
def package_verification(package):
    try:
        output = os.popen('pip list').read()
        return f'{package}' in output
    except Exception as e:
        print(f'Error en la verificación: {e}')
        return False

def package_installation(package_name):
    if package_verification(package_name):
        print(f"El paquete '{package_name}' ya está instalado. ")
    else:
        print('Instalando el paquete... ')
        try:
            os.system(f'pip install {package_name}')
            print(f'libreria {package_name} instalada correctamente! ')
        except Exception as e:
            print(f'Error en la instalación de {package_name}: {e}')

In [ ]:
package_installation('labelme') # Instalación de la herramienta de etiquetado Labelme

In [3]:
# Abrir la herramienta para el proceso de etiquetado 
try:
    subprocess.check_call(["labelme"]) 
except Exception as e:
    print(f'No fue posible abrir labelme: {e}')

In [ ]:
package_installation('labelme2yolo') # Instalación de la herramienta de conversión para el formato JSON de labelme a formato texto requerido por los modelos de detección de objetos YOLO.

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Conversión a formato YOLO</span>

In [22]:
# Todas las imagenes estan en una carpeta y no ordenadas en subcarpetas

dir_json_train='train' # Carpeta de entrenamiento, un 80% de imagenes.
dir_json_val='val' # Carpeta de validación, un 20% de imagenes.

# Agregar condicional para evitar dos conversiones
# Verificar si se puede tener un control de los archivos de entrenamiento nuevos
try:
    subprocess.check_call(["labelme2yolo", "--json_dir", f'{dir_train}\{dir_json_train}'])
    print(f'Conversión completada para la carpeta {dir_train}\{dir_json_train}')
    subprocess.check_call(["labelme2yolo", "--json_dir", f"{dir_train }\{dir_json_val}"])
    print(f'Conversión completada para la carpeta {dir_train }\{dir_json_val}')
    
except subprocess.CalledProcessError as e:
    print(f'Error al ejecutar función labelme2yolo: {e}')

Conversión completada para la carpeta c:\Users\matrix\pruebayolo\proyecto_yolo\dist\lote2_2\train
Conversión completada para la carpeta c:\Users\matrix\pruebayolo\proyecto_yolo\dist\lote2_2\val


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de recursos</span>

In [ ]:
package_installation('ultralytics') # Instalación de la biblioteca Ultralytics
#ultralytics.checks()

In [ ]:
#Función para determinar si tiene GPU Cuda para instalación de PYTorch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'CUDA: {torch.cuda.get_device_name(0)}')
else:
    try:
        command = 'wmic path win32_videocontroller get caption'
        device = subprocess.check_output(command, shell=True, universal_newlines=True)
        print(f'Tarjeta Grafica: {device.strip()}.\n \nAdvertencia: Debe utilizar la CPU para instalación de PyTorch! ')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar el comando: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Dirección para instalar mediante el comando la versión de PyTorch en función de los requerimientos computacionales: </span>[Versión PyTorch](https://pytorch.org/get-started/locally/)

In [ ]:
command_pytorch = 'pip3 install torch torchvision torchaudio' # Comando de instalación según la version personalizada a sus requerimientos computacionales # idear solucion mas amena al cliente
substrings = command_pytorch.split(" ")
try:
    subprocess.check_call(substrings)
except subprocess.CalledProcessError as e:
    print(f'Error al instalar la version de PyTorch: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Selección versión de modelo Yolo</span>

In [ ]:
package_installation('gdown')

In [4]:
def get_model_version():
    available_versions= ['n', 's', 'm', 'l', 'x'] # yolov8n.pt , yolov8s.pt , yolov8m.pt , yolov8l.pt , yolov8x.pt
    control= True
    while control:
        version_selection= input("Ingrese la versión del modelo que desea utilizar, entre las disponibles están: n, s, m, l, x. Advertencia: Si no desea instalar ninguna ingrese la palabra 'salir'! ") 
        if version_selection in available_versions:
            id_version= f'yolov8{version_selection}-seg.pt' 
            return id_version
        elif version_selection == "salir":
            control=False
            return None
        else:
            print('Version de modelo inexistente, ingrese nuevamente! ')
     
def model_installation(url, nombre_version):
    try:
        path_save = os.path.abspath(main_folder) + "\\dist\\model_version\\" + nombre_version
        if os.path.exists(path_save):
            print(f'\nSe está utilizando la versión "{nombre_version}" del modelo. ')
        else:
            url_installation = f'{url}/{nombre_version}'
            gdown.download(url_installation, path_save, quiet=False)
            print('Versión instalada correctamente!')
    except (NameError, UnboundLocalError) as e:
        print(f'Error en la instalación de la versión del modelo: {e}')
    return path_save

model_url= 'https://github.com/ultralytics/assets/releases/download/v0.0.0' 
model_version= get_model_version()
if model_version is not None:
    model_orig= model_installation(model_url, model_version) 
else:
    print('Ha salido con exito! ')


Se está utilizando la versión "yolov8m-seg.pt" del modelo. 


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">ENTRENAMIENTO</span>


In [7]:
model_segmentation=YOLO(model_orig)

PROJECT='segmentacion' # Permite asignar un nombre al directorio de inicio que contendrá los experimentos de segmentación de objetos, y debe estar entre comillas; se recomienda no utilsizar espacios en el nombre.
NAME='pruebaEntrenamiento' # El nombre del experimento entrenamiento para segmentación de instancias debe ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
TASK='segment' # Define la tarea principal que desea realizar con el modelo YOLO v8; en este caso, la segmentación implica el uso de máscaras para identificar objetos individuales en una imagen y segmentarlos del resto de la imagen.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada. Puede especificarlo como un número entero, como imgsz a 640 para obtener un cuadrado perfecto, o como una tupla, como imgsz=(640,480) para establecer dimensiones específicas de ancho y alto. Se recomienda ajustar este valor según el tamaño del objeto que desea segmentar. Para la segmentación de objetos pequeños, se recomienda aumentar el valor a más de 640 píxeles para obtener una resolución más alta.
DATA= dir_train_converted #  Le permite indicar la ruta al archivo que contiene los metadatos que utilizara el modelo segmentación de objetos y su configuración en formato YAML. Si especifica el valor data=None, el conjunto de datos coco128-seg.yaml se utiliza de forma predeterminada; de lo contrario, escriba la ruta al archivo YAML entre comillas utilizando barras diagonales (/) en lugar de barras invertidas (\).
EPOCHS=30 # Establezca el número de épocas del modelo YOLO v8 en la tarea de segmentación. Este valor representa el número total de iteraciones en todo el conjunto de datos de entrenamiento. Se recomienda experimentar con este parámetro dependiendo de la cantidad de imágenes disponibles. Si tiene un conjunto de datos grande, considere aumentar este valor por encima de 30 para obtener mejores resultados. Por otro lado, establecer epochs=None hará que el modelo continúe entrenándose hasta que la pérdida de validación deje de mejorar.
BATCH=-1 # Define la cantidad de imágenes procesadas simultáneamente en una iteración del modelo YOLO v8 en la segmentación. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
OPTIMIZER='auto' # Define el algoritmo de optimización para el modelo de segmentación YOLO. Su elección ajusta los pesos del modelo durante el entrenamiento y es crucial para la velocidad y rendimiento. Puede tomar valores como 'SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp' y 'auto', este último selecciona automáticamente el optimizador más adecuado a la tarea segmentación de objetos.
WORKERS=1 # Especifica la cantidad de subprocesos o núcleos utilizados para cargar datos en el modelo segmentación YOLO. Se recomienda ajustar la cantidad de subprocesos a la cantidad de núcleos de CPU disponibles en el sistema.
DEVICE= 'cpu' # Especifica el dispositivo de ejecución para la versión del modelo yolov8 en la operación de segmentación. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante el entrenamiento de segmentación de objetos. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
SAVE=False # Cuando se establece en True, el modelo guarda puntos de control periódicamente durante el entrenamiento de segmentación. Se recomienda tener cuidado al establecer este valor en Verdadero, ya que está relacionado con el hiperparámetro SAVE_PERIOD; si establece el valor del hiperparametro a False, la función Save_Period se desactivará.
SAVE_PERIOD=-1 # Se utiliza para especificar con qué frecuencia se guardan los puntos de control durante el entrenamiento de segmentación. Si se establece en un valor mayor que 0, el modelo guardará puntos de control cada número especificado de épocas. Sin embargo, si save_period se establece en -1, significa que la función esta deshabilitada.
PATIENCE=30 # Representa el número esperado de épocas durante el entrenamiento de segmentación. Si no se observa mejora en el conjunto de validación dentro de un período específico, se detiene el proceso. Esta técnica de parada temprana se utiliza para evitar el sobreajuste del modelo. Se recomienda ajustar este hiperparámetro en función de la duración esperada del entrenamiento.
VERBOSE=False # Se utiliza para controlar el número de impresiones durante la ejecución del entrenamiento de segmentación. Para suprimir la salida de información básica únicamente, debe establecer el valor del hiperparámetro en False, pero si desea una salida de progreso más detallada, establezca el valor en True.
RECT= False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de entrenamiento en la segmentación de instancias. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente lote.
COS_LR=False # Reemplaza el decaimiento escalonado predeterminado de YOLOv8, que reduce la tasa de aprendizaje en ciertas épocas, con el decaimiento escalonado cos_lr. Este ajusta la tasa de aprendizaje según las épocas restantes y la tasa de aprendizaje inicial, proporcionando una disminución más suave. Establezca este hiperparámetro en True para una reducción gradual de la tasa de aprendizaje, de lo contrario establezca en False.
FRACTION= 1.0 # Controla la fracción del conjunto de datos que se utilizara para el entrenamiento de segmentación. Debes establecer este parámetro entre 0.0 y 1.0. Por defecto, cuando es 1.0, se emplea el 100% de las imágenes disponibles en el conjunto de datos.
EXIST_OK=False # Controla la sobreescritura de un experimento de segmentación existente. Cuando se establece en False, el sistema no sobrescribirá, en su lugar devolverá una ruta incrementada. Esto es útil para prevenir la sobreescritura accidental de experimentos anteriores. Para activar la función, asigna el valor True.
OVERLAP_MASK=True # Determina si las máscaras que representan áreas de interés en la imagen deben superponerse. Establecer el valor del hiperparametro a True permite que estas máscaras compartan áreas, lo que significa que los límites de los objetos en la imagen puede coincidir parcialmente. Esta opción ahorra memoria, acelera el entrenamiento, y es especialmente útil para grandes conjuntos de datos o modelos complejos.
MASK_RATIO=1 # Configure la reducción de muestreo para la máscara de segmentación. Se sugiere fijar el valor en el rango de 1 para entrenamiento en resolución nativa, ideal para aplicaciones que demandan alta precisión, hasta 4 para una reducción en un factor de 4, acelerando el entrenamiento con una precisión aceptable y ahorrando memoria. Este hiperparámetro solo acepta valores enteros.

model_segmentation.train(project=PROJECT,name=NAME, task=TASK, data=DATA, imgsz=IMGSZ, epochs=EPOCHS, batch=BATCH, workers=WORKERS, device=DEVICE, plots=PLOTS, verbose=VERBOSE, rect=RECT, cos_lr=COS_LR, optimizer=OPTIMIZER, patience=PATIENCE, exist_ok=EXIST_OK, overlap_mask=OVERLAP_MASK, mask_ratio=MASK_RATIO, fraction=FRACTION)

New https://pypi.org/project/ultralytics/8.1.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
engine\trainer: task=segment, mode=train, model=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\model_version\yolov8n-seg.pt, data=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\data\dataset.yaml, epochs=30, patience=30, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=1, project=segmentacion, name=pruebaEntrenamiento, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=1, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fal

train: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data\train\labels.cache... 199 images, 0 backgrounds, 0 corrupt: 100%|██████████| 199/199 [00:00<?, ?it/s]
val: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data\val\labels.cache... 48 images, 0 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<?, ?it/s]


Plotting labels to segmentacion\pruebaEntrenamiento\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to segmentacion\pruebaEntrenamiento
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/30         0G      2.121      4.484      3.576      1.757        100        640: 100%|██████████| 13/13 [02:32<00:00, 11.73s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:10<00:00,  5.45s/it]

                   all         48        234    0.00564      0.445     0.0381     0.0123    0.00267      0.238    0.00613    0.00154


...

                   all         48        234      0.278      0.289      0.193     0.0922      0.232      0.146       0.12     0.0462



30 epochs completed in 1.231 hours.
Optimizer stripped from segmentacion\pruebaEntrenamiento\weights\last.pt, 6.8MB
Optimizer stripped from segmentacion\pruebaEntrenamiento\weights\best.pt, 6.8MB

Validating segmentacion\pruebaEntrenamiento\weights\best.pt...
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:09<00:00,  4.94s/it]


                   all         48        234      0.279      0.287      0.193     0.0923      0.234      0.146       0.12     0.0462
Speed: 3.1ms preprocess, 110.5ms inference, 0.0ms loss, 7.0ms postprocess per image
Results saved to segmentacion\pruebaEntrenamiento


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002B1CFD27040>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">Configuración para la exportación</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 18px;">A continuación se muestra una tabla de referencia para exportar un modelo YOLOv8 entrenado en la tarea segmentación de instancias. Tenga en cuenta que configurar el parámetro de `format` es fundamental para el proceso de exportación. Antes de continuar, asegúrese de verificar y ajustar estos valores a sus requisitos específicos: </span>

| Formatos                                                             | Asignación | Extensión                     | Hyperparámetros                                            | Descripción                                        |
|--------------------------------------------------------------------|-------------------|---------------------------|-----------------------------------------------------|----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | -                                                   | Modelo en formato PyTorch                           |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | "torchscript"     | `yolov8n.torchscript`     | `imgsz`, `optimize`                                 | Simplifica la implementación de modelos PyTorch en entornos de producción y aplicaciones eficientes, mejorando la portabilidad y el rendimiento al permitir ejecutar una representación intermedia en entornos sin Python.                       |
| [ONNX](https://onnx.ai/)                                           | "onnx"            | `yolov8n.onnx`            | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     | Desarrollado para promover la interoperabilidad, la optimización del hardware y la colaboración entre comunidades, al tiempo que responde a la necesidad de portabilidad de los modelos entre distintos marcos y herramientas de aprendizaje automático.                              |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | "openvino"        | `yolov8n_openvino_model/` | `imgsz`, `half`, `int8`                             | Elaborado para promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware Intel.                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | "engine"          | `yolov8n.engine`          | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` | Permite promover la interoperabilidad, la optimización del hardware y la implantación eficiente de modelos en distintos marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de NVIDIA.                     |
| [CoreML](https://github.com/apple/coremltools)                     | "coreml"          | `yolov8n.mlpackage`       | `imgsz`, `half`, `int8`, `nms`                      | Posibilita promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de Apple.                            |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | "saved_model"     | `yolov8n_saved_model/`    | `imgsz`, `keras`, `int8`                            | Empleado para guardar, compartir y desplegar modelos entrenados con TensorFlow. Versátil y facilita el despliegue en diversas plataformas como servidores, dispositivos móviles, embebidos y navegadores.                     |
| [TF Lite](https://www.tensorflow.org/lite)                         | "tflite"          | `yolov8n.tflite`          | `imgsz`, `half`, `int8`                             | Diseñado para el aprendizaje automático en dispositivos, TF Lite aborda restricciones clave como latencia, privacidad, conectividad, tamaño y consumo de energía. Es esencial para desplegar modelos en dispositivos móviles e integrados, ofreciendo una solución ligera y eficiente.                           |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | "edgetpu"         | `yolov8n_edgetpu.tflite`  | `imgsz`                                             | Utilizado para desplegar modelos de aprendizaje automático en el Edge TPU de TensorFlow. El Edge TPU es un pequeño ASIC (Circuito Integrado Específico de Aplicación) diseñado por Google para ofrecer inferencias de aprendizaje automático de alto rendimiento en dispositivos de bajo consumo.                       |
| [TF.js](https://www.tensorflow.org/js)                             | "tfjs"            | `yolov8n_web_model/`      | `imgsz`                                             | Facilita el despliegue de modelos de aprendizaje automático en navegadores web y Node.js, destacando la portabilidad y la facilidad de uso.                    |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | "paddle"          | `yolov8n_paddle_model/`   | `imgsz`                                             | Utilizado para desplegar modelos en PaddlePaddle, una plataforma de aprendizaje profundo de código abierto, paralela y distribuida que tiene su origen en la práctica industrial.                      |
| [ncnn](https://github.com/Tencent/ncnn)                            | "ncnn"            | `yolov8n_ncnn_model/`     | `imgsz`, `half`                                     | Formato optimizado para plataformas móviles, ofreciendo alto rendimiento. Puede incluir una estructura de archivo de modelo con información sobre capas, blobs de entrada y salida, y otros parámetros.                              |


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">EXPORTACIÓN</span>


In [ ]:
selected_model_seg=YOLO(model_train)

FORMAT='onnx' # Seleccione el formato de exportación del modelo segmentación, empleando la tabla previamente proporcionada; en la columna "Asignación" se indican las opciones para ajustar este valor.
INT8=True # Establezca este parámetro en True al utilizar la CPU y en False en caso contrario. La cuantificación a INT8 mejora la eficiencia del modelo segmentación en cuanto a memoria y velocidad de inferencia, especialmente en hardware que admite esta precisión.
HALF=False # Configúrelo en True cuando use la GPU; en caso contrario, False. La cuantificación a FP16 mejora la eficiencia de la memoria del modelo segmentación y la velocidad de inferencia, especialmente en hardware que admite precisión de punto flotante de 16 bits.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada para la exportación del modelo de segmentación. Puede especificarlo como un número entero, por ejemplo, 640 para un cuadrado perfecto, o como una tupla, por ejemplo, (640, 480) para dimensiones específicas de ancho y alto. Las imágenes que ingreses al modelo después de la exportación deben tener las mismas dimensiones específicas que has configurado para adaptarse a los requisitos del escenario de despliegue.
OPTIMIZE=False # Controla la optimización en modelos de segmentación de instancias a TorchScript para su implementación móvil. Es importante destacar que esta función puede resultar en un aumento significativo en el tamaño del modelo exportado, lo cual puede no ser ideal para aplicaciones móviles. Se configura con True para activar y False para desactivar.
DYNAMIC=False # Controla la habilitación de ejes dinámicos en modelos de segmentación, lo cual es particularmente útil para gestionar tamaños de lote variables. Esta característica funciona bien en escenarios donde el tamaño del lote puede cambiar durante la inferencia, como aplicaciones en tiempo real o de transmisión por secuencias. Los valores aceptados son Verdadero para habilitar la función y Falso para deshabilitarla.
SIMPLIFY=False # En la exportación de modelos de segmentación a ONNX|TensorRT, este hiperparámetro personaliza la complejidad del modelo, optimizando, eliminando capas redundantes y reduciendo la precisión de los parámetros. Se activa con True y se desactiva con False.
OPSET=None # Especifica la versión del conjunto de operadores en ONNX al exportar el modelo segmentación desde marcos como PyTorch o TensorFlow. Si se deja en "None", ONNX utilizará automáticamente la versión más reciente disponible; para una versión específica, asigne el número entre comillas, por ejemplo, "11".
WORKSPACE=4 # En la exportación de modelos de segmentación a TensorRT, establece el tamaño del espacio de trabajo en GB asignado para optimizar y preparar el modelo de red neuronal. Este espacio se utiliza durante el proceso de construcción del motor para lograr una ejecución eficiente en hardware GPU mediante la biblioteca TensorRT.
NMS=False # En la exportación de modelos de segmentación a CoreML, controla la inclusión de la Supresión No Máxima (NMS) en el modelo para eliminar cuadros delimitadores redundantes en la segmentación de instancias y mejorar la precisión de las predicciones. Establecer 'NMS' en 'False' ignora NMS en los modelos CoreML exportados.  Este ajuste, configurable durante la exportación del modelo YOLO, lo que permite a los usuarios optimizar la implementación del modelo en una variedad de plataformas y dispositivos.
KERAS= False # En la exportación de modelos de segmentación a TF SavedModel y TF Lite, permite optimizar el despliegue en diversas plataformas y dispositivos. Incluye también el formato del archivo, el dispositivo de ejecución y la posibilidad de manejar múltiples etiquetas por caja. Establezca el valor del hiperparámetro en True si está familiarizado con Keras; de lo contrario, en False para excluir su uso en la exportación.

selected_model_seg.export(format=FORMAT, imgsz=IMGSZ, dynamic=DYNAMIC, simplify=SIMPLIFY, opset=OPSET)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">VALIDACIÓN</span>


In [ ]:
selected_model_seg=YOLO(model_train)

NAME='pruebaValidacionSeg' # El nombre del experimento de validación para segmentación de instancias deben ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
DATA= dir_train_converted # Permite indicar la ruta al archivo que contiene los metadatos necesarios para el proceso de validación, la ruta debe ser proporcionada entre comillas.
SAVE_JSON=True # Si se configura como True, habilita la funcionalidad de guardar los resultados obtenidos de manera detallada del proceso de validación en un formato estructurado JSON.
IMGSZ=640 # Establece las dimensiones en píxeles de la imagen de entrada para la validación del modelo de segmentación. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda usar el mismo valor que utilizo durante el entrenamiento del modelo.
BATCH=16 # Define la cantidad de imágenes procesadas simultáneamente en una iteración para la validación de segmentos. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
SAVE_HYBRID=True # Activa la función con True para guardar una versión híbrida de la etiqueta, incluyendo la original y predicciones adicionales. Útil para el análisis detallado del rendimiento del modelo segmentación durante la validación; establezca en False para mostrar solo las predicciones.
CONF=0.5 # Establece el umbral de confianza para la validación de clases en la tarea de segmentación. Se recomienda un valor entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión. 
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara segmentar en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
DEVICE='CPU' # Especifica el dispositivo de ejecución para la prueba de validación en la operación de segmentación. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante la validación en la segmentación de instancias. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
RECT=False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de validación en la segmentación de instancias. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente.
IOU=0.6 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,6. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren el mismo segmento. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.

selected_model_seg.val(name=NAME, data=DATA, save_json=SAVE_JSON, imgsz=IMGSZ, batch=BATCH, save_hybrid=SAVE_HYBRID, conf=CONF, max_det=MAX_DET, device=DEVICE, plots=PLOTS, rect=RECT, iou=IOU)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de fuentes</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Para utilizar múltiples fuentes de datos al realizar predicciones con el modelo, se requiere que se ajuste el parámetro'source' a sus necesidades, tal como se indica en la siguiente tabla:</span>


| Fuentes          | Asignación                             | Tipo             | Notas                                                           |
| --------------- | ------------------------------------ | ----------------- | --------------------------------------------------------------- |
| `image`           | 'image.jpg'                          | str or Path       | Archivo que contiene una única imagen.                                              |
| `URL`             | 'https://ultralytics.com/images/bus.jpg' | str               | Dirección que especifica la ubicación de una imagen en la web.                                                 |
| `screenshot`      | 'screen'                             | str               | El sistema captura la imagen actualmente visible en la pantalla y la utiliza como entrada para el modelo.                                           |
| `PIL`             | Image.open('im.jpg')                 | PIL.Image         | Utilizado para cargar imágenes en formato HWC (altura, ancho, canales) con canales RGB (rojo, verde y azul) mediante la biblioteca Python Imaging Library (PIL).                                   |
| `OpenCV`          | cv2.imread('im.jpg')                 | np.ndarray        | Permite la lectura de una imagen desde un archivo en formato HWC con canales BGR (azul, verde, rojo) utilizando la biblioteca OpenCV, almacenando la imagen como un array de NumPy.                    |
| `numpy`           | np.zeros((640,1280,3))               | np.ndarray        | Genera un array de ceros con las dimensiones especificadas para un formato HWC con canales BGR, utilizando la biblioteca NumPy.                    |
| `torch`           | torch.zeros(16,3,320,640)            | torch.Tensor      | Crea un tensor de ceros con las dimensiones especificadas para un formato HWC con canales RGB, empleando el framework PyTorch.               |
| `CSV`             | 'sources.csv'                        | str or Path       | Archivo de texto que almacena las rutas a las imágenes que se procesarán.   |
| `video`          | 'video.mp4'                          | str or Path       | Proporciona acceso a un archivo de video único.                       |
| `directory`      | 'path/'                              | str or Path       | Directorio que contiene múltiples archivos de imagen.               |
| `glob`           | 'path/*.jpg'                         | str               | Permite acceder a varias imágenes en un directorio usando expresiones de coincidencia de patrones. |
| `YouTube`        | 'https://youtu.be/LNwODJXcvt4'       | str               | Facilita el acceso a videos desde la plataforma YouTube.                                         |
| `stream`         | 'rtsp://example.com/media.mp4'      | str               | Permite la conexión a flujos de video o audio en tiempo real mediante protocolos como RTSP, RTMP, TCP o IP, ya sea a través de internet o una red local. |
| `multi-stream`   | 'list.streams'                       | str or Path       | Se utiliza para transmitir varios flujos de medios simultáneamente, permitiendo el procesamiento y análisis paralelo de múltiples flujos de medios. |

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Formatos para las imágenes: </span>

| Image Suffixes | Reference                           |
| --------------- | ----------------------------------- |
| .bmp            | [Microsoft BMP File Format](https://docs.fileformat.com/es/image/bmp/)           |
| .dng            | [Adobe DNG](https://docs.fileformat.com/es/image/dng/)                           |
| .jpeg           | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .jpg            | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .mpo            | [Multi Picture Object](https://docs.fileformat.com/es/image/mpo/)                |
| .png            | [Portable Network Graphics](https://docs.fileformat.com/es/image/png/)           |
| .tif            | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .tiff           | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .webp           | [WebP](https://docs.fileformat.com/es/image/webp/)                                |
| .pfm            | [Portable FloatMap](https://docs.fileformat.com/font/pfm/)                   |


<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Formatos para los videos: </span>

| Video Suffixes | Reference                           |
| -------------- | ----------------------------------- |
| .asf           | [Advanced Systems Format](https://docs.fileformat.com/es/video/asf/)             |
| .avi           | [Audio Video Interleave](https://docs.fileformat.com/es/video/avi/)              |
| .gif           | [Graphics Interchange Format]()          |
| .m4v           | [MPEG-4 Part 14](https://docs.fileformat.com/es/video/m4v/)                      |
| .mkv           | [Matroska](https://docs.fileformat.com/es/video/mkv/)                            |
| .mov           | [QuickTime File Format](https://docs.fileformat.com/es/video/mov/)               |
| .mp4           | [MPEG-4](https://docs.fileformat.com/es/video/mp4/)          |
| .mpeg          | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .mpg           | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .ts            | [MPEG Transport Stream](https://docs.fileformat.com/es/video/ts/)               |
| .wmv           | [Windows Media Video](https://docs.fileformat.com/es/video/wmv/)                 |
| .webm          | [WebM Project](https://docs.fileformat.com/es/video/webm/)                        |


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">PREDICCIÓN</span>


In [4]:
SOURCE= dir_prueba # Establezca el origen de datos que el modelo de detección utilizará para realizar predicciones. Configure el valor de este hiperparámetro según la tabla proporcionada anteriormente.
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara predecir en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada durante la predicción en tareas de detección. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda utilizar los mismos valores utilizados durante el entrenamiento del modelo para mantener la coherencia en la inferencia.
CONF=0.5 # Establece el umbral de confianza durante el proceso de predicción en la tarea de detección. Se recomienda establecer el valor hiperparámetro entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión en la inferencia.
LINE_WIDTH= None # Determina el grosor en píxeles de los cuadros delimitadores que rodean los objetos detectados por el modelo. Puede establecer el grosor de la línea como un número entero en el que, a mayor valor, la línea será más gruesa, también puede utilizar como valor None para que el grosor se ajuste de forma automatizada, proporcionando una línea proporcional al tamaño de la imagen.
VISUALIZE=False # Determina si las características del modelo de detección deben mostrarse durante la predicción. Establecer esto en True permite que las características se muestren como mapas intermedias, lo que hace que el modelo sea más fácil de entender. Si se establece en False, no se mostrarán las características del modelo. 
IOU=0.7 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,7. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren la misma detección. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.
DEVICE='cpu' # Especifica el dispositivo de ejecución para la prueba de predicción en la operación de detección. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
VID_STRIDE=False # Controla la velocidad de los fotogramas durante el proceso de predicción en vídeos o secuencias de tiempo real. Al establecerlo en True el modelo se adapta a la velocidad de fotogramas especificada por la fuente de vídeo, procesando cada fotograma individualmente. Para desactivar esta función indique como valor False.
STREAM_BUFFER=False # Controla el almacenamiento en búfer de los fotogramas para la detección. Si es True, se almacenan todos los fotogramas para el procesamiento en tiempo real de vídeos o transmisiones en directo; si es False, devuelve el fotograma más reciente.
SAVE_FRAMES=False # Controla la captura y almacenamiento de los fotogramas predichos por el modelo de detección. Con True, se guardarán todos los fotogramas individuales predichos; con False, no se realizará el almacenamiento de los fotogramas.
AUGMENT=False # Aplica transformaciones a las imágenes de entrada, tales como giros, rotaciones, recortes y cambios de color, para diversificar los datos y mejorar la predicción en la detección. Establecer en True para activar la función, False para desactivar.
SAVE_CROP=False # Determina si se deben guardar imágenes recortadas con los resultados durante la predicción en la detección. Al establecerlo en "False", las imágenes recortadas no se guardarán, lo que reduce el tamaño del archivo. Con el valor "True", se guardarán las imágenes recortadas correspondientes a las áreas detectadas.
SHOW=False # Determina si se deben mostrar las imágenes o vídeos detectados durante la predicción. Al establecerlo en "True", permite la visualización de las predicciones en el mismo entorno, proporcionando una representación visual de los resultados. Si se establece en "False", las predicciones no se mostrarán. 

def train_classes(model_preentrenado, model_original):
    try:
        selected_model_det=YOLO(model_preentrenado)
        
        resultado_rayon= selected_model_det.predict(name='PruebaPrediccion_rayon', source=SOURCE,conf=CONF, save_txt=True, max_det=MAX_DET, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, classes=0,  save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save=True, save_conf=True) 
        
        resultado_abolladura= selected_model_det.predict(name='PruebaPrediccion_abolladura', source=SOURCE,conf=CONF, save_txt=True, max_det=MAX_DET, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, classes=1,  save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save=True, save_conf=True)
        
        if resultado_rayon or resultado_abolladura is not None:
            
            selected_model_det_car=YOLO(model_original)
            resultado_carro= selected_model_det_car.predict(name='PruebaPrediccion_car', source=SOURCE,conf=CONF, save_txt=True, max_det=MAX_DET, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, classes=2,  save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save=True, save_conf=True) 
            
        return resultado_rayon, resultado_abolladura, resultado_carro
    
    except Exception as e:
        print(f'Error en el entrenamiento del modelo: {e}')

model_orig='C:/Users/matrix/pruebayolo/proyecto_yolo/dist/model_version/yolov8m-seg.pt'
results_rayon, results_abolladura, results_car= train_classes(model_train, model_orig)


image 1/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\20170531_010133.jpg: 384x640 (no detections), 203.8ms
image 2/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\OIP (1).jpg: 480x640 4 rayons, 123.1ms
image 3/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\R.jpg: 384x640 (no detections), 79.0ms
image 4/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\abolladura-en-un-coche-632c58a6e2ea5.jpg: 448x640 (no detections), 120.4ms
image 5/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\abolladuraautos-fd03b3bae67d7363172c41cc00198c51-1200x800.jpg: 448x640 (no detections), 85.0ms
image 6/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\big_ray.jpg: 320x640 (no detections), 86.0ms
image 7/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\coche-después-de-una-ruina-la-necesidad-ser-reparado-esperándola-es-demanda-de-seguro-29745503.jpg: 448x640 (no detections), 103.0ms
image 8/9 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets\rayonesauto-e68618ca9c3

In [11]:
for i in results_rayon:
    #if i.masks is not None:
    print(i.masks)

None
ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
        

In [12]:
for i in results_car:
    print(i.masks)

None
None
None
None
ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
orig_shape: (800, 1200)
shape: torch.Size([1, 448, 640])
xy: [array([[     333.75,         2.5],
       [     331.87,       4.375],
       [     296.25,       4.375],
       ...,
       [     1186.9,       4.375],
       [     1158.8,       4.375],
       [     1156.9,         2.5]], dtype=float32)]
xyn: [array([[    0.27812,    0.003125],
       [    0.27656,   0.0054687],
       [    0.24687,   0.0054687],
       ...,
       [    0.98906,   0.0054687],
       [    0.96562,   0.0054687],
       [    0.96406,    0.003125]], dtype=float32)]
ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
   

In [13]:
for i in results_abolladura:
    print(i.masks)

None
None
None
None
None
None
None
None
ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
orig_shape: (675, 1013)
shape: torch.Size([1, 448, 640])
xy: [array([[     742.34,      288.43],
       [     740.76,      290.02],
       [     726.51,      290.02],
       [     724.93,       291.6],
       [     720.18,       291.6],
       [     717.01,      294.76],
       [     717.01,      296.35],
       [     715.43,      297.93],
       [     715.43,      299.51],
       [     713.85,       301.1],
       [     713.85,      302.68],
       [     685.36,      331.17],
       [     683.78,      331.17],
       [     683.78,      332.75],
       [     679.03,       337.5],
       [     677.44,       337.5],
       [    

In [5]:
def calcular_area(abolladura):
    sum_area=0
    for id, i in enumerate(abolladura):
        print(f'imagen {id + 1}')
        if i.masks is not None:
            mascara= i.masks.xy 
            for l, e in enumerate(mascara):
                x= e[:, 0]
                y= e[:, 1] 
                area = 0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1))) 
                sum_area += area
                print(f'Mascara {l + 1}: {area}')
        else: 
            print('No hay detecciones ')
    return sum_area

calcular_area(results_rayon) #

imagen 1
No hay detecciones 
imagen 2
Mascara 1: 98.75
Mascara 2: 230.25
Mascara 3: 317.0
Mascara 4: 87.4921875
imagen 3
No hay detecciones 
imagen 4
No hay detecciones 
imagen 5
No hay detecciones 
imagen 6
No hay detecciones 
imagen 7
No hay detecciones 
imagen 8
Mascara 1: 303.4375
imagen 9
No hay detecciones 


1036.9296875

In [18]:
# areas totales 
def calcular_area(abolladura):
    sum_area=0
    for i in abolladura:
        if i.masks is not None:
            mascara= i.masks.xy 
            for e in mascara:
                x= e[:, 0]
                y= e[:, 1] 
                area = 0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1))) 
                sum_area += area
    return sum_area

def procesar_deteccion_principal(abolladura, rayon, results_car):
    area_carro = 0
    for id_imagen, e_abolladura in enumerate(abolladura):
        print(f'imagen {id_imagen + 1}: ')
        instancia_img = e_abolladura.path
        area_abolladura = calcular_area(e_abolladura)
        print(f'area abolladura: {area_abolladura}px^2')
        
        area_rayon = 0
        for e_rayon in rayon:
            if e_rayon.path == instancia_img:
                area_rayon = calcular_area(e_rayon)
                print(f'area rayon: {area_rayon}px^2')

        if area_abolladura > 0 or area_rayon > 0:
            procesar_deteccion_segundaria(results_car, area_abolladura, area_rayon, instancia_img)
        else:
            print(f'area carro {area_carro}px^2')

def procesar_deteccion_segundaria(carro, area_abolladura, area_rayon, instancia_img): 
    area_total_afectada = 0   
    sum_area_principal= 0
    tipo_afectacion= ''
    
    for i in carro:   
        if i.path == instancia_img:
            area_carro = calcular_area(i)
            print(f'area carro: {area_carro}px^2')
            if area_abolladura > 0:
                area_total_afectada = abs(area_carro - area_abolladura)
                tipo_afectacion = 'abolladura'
            elif area_rayon > 0:
                area_total_afectada = abs(area_carro - area_rayon)
                tipo_afectacion = 'rayon'
            elif area_abolladura > 0 and area_rayon > 0:
                sum_area_principal = area_abolladura + area_rayon
                area_total_afectada = abs(area_carro - sum_area_principal)
                tipo_afectacion = 'rayon_abolladura'
                
    print(f'area afectada en el carro por {tipo_afectacion}: {area_total_afectada}px^2')
                
            
procesar_deteccion_principal(results_abolladura, results_rayon, results_car)

imagen 1: 
area abolladura: 0px^2
area rayon: 0px^2
area carro 0px^2
imagen 2: 
area abolladura: 0px^2
area rayon: 733.4921875px^2
area carro: 0px^2
area afectada en el carro por rayon: 733.4921875px^2
imagen 3: 
area abolladura: 0px^2
area rayon: 0px^2
area carro 0px^2
imagen 4: 
area abolladura: 0px^2
area rayon: 0px^2
area carro 0px^2
imagen 5: 
area abolladura: 0px^2
area rayon: 0px^2
area carro 0px^2
imagen 6: 
area abolladura: 0px^2
area rayon: 0px^2
area carro 0px^2
imagen 7: 
area abolladura: 0px^2
area rayon: 0px^2
area carro 0px^2
imagen 8: 
area abolladura: 0px^2
area rayon: 303.4375px^2
area carro: 367066.0px^2
area afectada en el carro por rayon: 366762.5625px^2
imagen 9: 
area abolladura: 66108.0px^2
area rayon: 0px^2
area carro: 0px^2
area afectada en el carro por abolladura: 66108.0px^2


In [6]:
# areas individuales 
def calcular_area(mascara, tipo_afectacion= None):
    sum_area = 0
    if mascara is not None:
        print(f'Mascaras {tipo_afectacion}: ')
        for id, e in enumerate(mascara):
            x = e[:, 0]
            y = e[:, 1] 
            area = 0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
            sum_area += area
            print(f'Mascara {id + 1}: {np.round(area).astype(int)}')
        return sum_area

def buscar_area_por_instancia(results, instancia_img, tipo_afectacion):
    for i in results:
        if i.path == instancia_img:
            return calcular_area(i.masks.xy, tipo_afectacion) if i.masks is not None else 0

def procesar_deteccion_principal(abolladura, rayon, results_car):
    for id_imagen, e_abolladura in enumerate(abolladura):
        print(f'\nimagen {id_imagen + 1}: ')
        instancia_img = e_abolladura.path

        area_abolladura = buscar_area_por_instancia(abolladura, instancia_img, tipo_afectacion='abolladura')
        print(f'area abolladura: {round(area_abolladura)} px^2')

        area_rayon = buscar_area_por_instancia(rayon, instancia_img, tipo_afectacion='rayon')
        print(f'area rayon: {round(area_rayon)} px^2')

        if area_abolladura > 0 or area_rayon > 0:
            procesar_deteccion_segundaria(results_car, area_abolladura, area_rayon, instancia_img)
        else:
            print(f'area carro: 0px^2')

def procesar_deteccion_segundaria(carro, area_abolladura, area_rayon, instancia_img):
    area_carro = buscar_area_por_instancia(carro, instancia_img, tipo_afectacion='carro')
    print(f'area carro: {round(area_carro)} px^2')

    area_total_afectada = abs(area_carro - area_abolladura) if area_abolladura > 0 else 0
    area_total_afectada = abs(area_carro - area_rayon) if area_rayon > 0 else area_total_afectada
    if area_abolladura > 0 and area_rayon > 0:
        sum_area_principal = area_abolladura + area_rayon 
        area_total_afectada = sum_area_principal - area_carro

    print(f'area afectada en el carro: {round(area_total_afectada)} px^2')

procesar_deteccion_principal(results_abolladura, results_rayon, results_car)



imagen 1: 
area abolladura: 0 px^2
area rayon: 0 px^2
area carro: 0px^2

imagen 2: 
area abolladura: 0 px^2
Mascaras rayon: 
Mascara 1: 99
Mascara 2: 230
Mascara 3: 317
Mascara 4: 87
area rayon: 733 px^2
area carro: 0 px^2
area afectada en el carro: 733 px^2

imagen 3: 
area abolladura: 0 px^2
area rayon: 0 px^2
area carro: 0px^2

imagen 4: 
area abolladura: 0 px^2
area rayon: 0 px^2
area carro: 0px^2

imagen 5: 
area abolladura: 0 px^2
area rayon: 0 px^2
area carro: 0px^2

imagen 6: 
area abolladura: 0 px^2
area rayon: 0 px^2
area carro: 0px^2

imagen 7: 
area abolladura: 0 px^2
area rayon: 0 px^2
area carro: 0px^2

imagen 8: 
area abolladura: 0 px^2
Mascaras rayon: 
Mascara 1: 303
area rayon: 303 px^2
Mascaras carro: 
Mascara 1: 367066
area carro: 367066 px^2
area afectada en el carro: 366763 px^2

imagen 9: 
Mascaras abolladura: 
Mascara 1: 66108
area abolladura: 66108 px^2
area rayon: 0 px^2
area carro: 0 px^2
area afectada en el carro: 66108 px^2


# Función de área para mascaras segmentadas

In [3]:
# areas totales dataframe
def calculate_area(vertices): 
    total_area_mask = 0 # Se inicializa una variable acumuladora para almacenar la suma total del área de cada máscara en una imagen.
    for point in vertices:
        coordinate_x = point[:, 0] # Asigna todas filas correspondientes a la primer columna como coordenada X del formato masks que entrega por medio de arrays los vertices (x , y) de los puntos de contorno por medio de una lista para cada región segmentada.
        coordinate_y = point[:, 1] # Asigna todas las filas correspondientes a la segunda columna como coordenada Y.
        mask_area = np.round(0.5 * abs(np.dot(coordinate_x, np.roll(coordinate_y, 1)) - np.dot(coordinate_y, np.roll(coordinate_x, 1)))).astype(int) # Almacena el resultado del area x mascara, basandose en la formula Shoelace o lazada de Gauss. Utiliza la suma y la resta de los productos cruzados de las coordenadas de los vertices del poligono; el termino np.roll(y, 1) y np.roll(x, 1) crean versiones desplazadas circularmente, es esencial para que np.dot() calcule el producto punto de los vertices del poligono, la diferencia entre los productos cruzados se multiplicara por 1/2 tomando el valor absoluto.
        total_area_mask += mask_area
    return total_area_mask 

def search_for_mask(segmentation_results, image_instance):
    for attribute in segmentation_results: # Itera a través de los atributos en los resultados de segmentación.
        if attribute.path == image_instance: # Comprueba si la ruta de la instancia actual coincide con la imagen en cada clase.
            return calculate_area(attribute.masks.xy) if attribute.masks is not None else 0 # Retorna el área calculada en píxeles solo si la máscara no está vacía.

def process_main_detection(abolladura_class, rayon_class, car_results):
    data = {'imagen':[], 'area abolladura':[], 'area rayon':[], 'area carro':[], 'afectacion':[]} # Se define la estructura inicial del dataframe, el orden de las columnas y su información en listas que están inicialmente vacías.
    
    for id_imagen, result in enumerate(abolladura_class): #
        control_image = result.path #

        data['imagen'].append(id_imagen + 1) # 
        
        area_abolladura = search_for_mask(abolladura_class, control_image) #
        data['area abolladura'].append(f'{area_abolladura}px^2')
        
        area_rayon = search_for_mask(rayon_class, control_image)
        data['area rayon'].append(f'{area_rayon}px^2')

        if area_abolladura > 0 or area_rayon > 0: # 
            car_masks= process_car_detection(car_results, area_abolladura, area_rayon, control_image) 
            data['area carro'].append(f'{car_masks["area carro"]}px^2')
            data['afectacion'].append(f'{car_masks["afectacion"]}px^2')
        else:
            data['area carro'].append('0px^2')
            data['afectacion'].append('0px^2')
    
    df= pd.DataFrame(data)
    return df

def process_car_detection(carro, area_abolladura, area_rayon, instancia_img):
    area_carro = search_for_mask(carro, instancia_img)

    area_total_afectada = abs(area_carro - area_abolladura) if area_abolladura > 0 else 0
    area_total_afectada = abs(area_carro - area_rayon) if area_rayon > 0 else area_total_afectada
    if area_abolladura > 0 and area_rayon > 0:
        sum_area_principal = area_abolladura + area_rayon 
        area_total_afectada = sum_area_principal - area_carro
    return {'area carro': area_carro, 'afectacion': area_total_afectada}

dataframe=process_main_detection(results_abolladura, results_rayon, results_car)
estilo= dataframe.style.set_properties(**{'text-align': 'rigth'})
estilo

,imagen,area abolladura,area rayon,area carro,afectacion
0,1,0px^2,0px^2,0px^2,0px^2
1,2,0px^2,733px^2,0px^2,733px^2
2,3,0px^2,0px^2,0px^2,0px^2
3,4,0px^2,0px^2,0px^2,0px^2
4,5,0px^2,0px^2,0px^2,0px^2
5,6,0px^2,0px^2,0px^2,0px^2
6,7,0px^2,0px^2,0px^2,0px^2
7,8,0px^2,527px^2,0px^2,527px^2
8,9,0px^2,303px^2,367066px^2,366763px^2
9,10,66108px^2,0px^2,0px^2,66108px^2


In [11]:
# idea 1
# areas individuales dataframe
def calcular_area(mascara):
    sum_area = 0
    areas=[]
    if mascara is not None:
        for id, e in enumerate(mascara):
            x = e[:, 0]
            y = e[:, 1] 
            area = np.round(0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))).astype(int)
            areas.append(area) #
            sum_area += area
            
        return sum_area, areas

def buscar_area_por_instancia(results, instancia_img):
    for i in results:
        if i.path == instancia_img:
            return calcular_area(i.masks.xy) if i.masks is not None else (0, [])

def procesar_deteccion_principal(abolladura, rayon, results_car):
    data_main = {'imagen':[], 'mascara abolladura':[], 'area abolladura':[], 'mascara rayon':[] ,'area rayon':[], 'mascara carro':[],'area carro':[], 'afectacion':[]}
    
    for id_imagen, e_abolladura in enumerate(abolladura):
        instancia_img = e_abolladura.path

        data_main['imagen'].append(id_imagen + 1)
        area_abolladura, mascara_abolladura = buscar_area_por_instancia(abolladura, instancia_img)
        mascaras_abolladuras = procesar_dataframe(mascara_abolladura)
        data_main['mascara abolladura'].append(f'{mascaras_abolladuras}px^2')
        data_main['area abolladura'].append(f'{area_abolladura}px^2')
        
        area_rayon, mascara_rayon = buscar_area_por_instancia(rayon, instancia_img)
        mascaras_rayones=procesar_dataframe(mascara_rayon)
        data_main['mascara rayon'].append(f'{mascaras_rayones}px^2')
        data_main['area rayon'].append(f'{area_rayon}px^2')
        
        if area_abolladura > 0 or area_rayon > 0:
            resultado_carro=procesar_deteccion_segundaria(results_car, area_abolladura, area_rayon, instancia_img)
            mascaras_carros=procesar_dataframe(resultado_carro["mascara carro"])
            data_main['mascara carro'].append(f'{mascaras_carros}px^2')
            data_main['area carro'].append(f'{resultado_carro["area carro"]}px^2')
            data_main['afectacion'].append(f'{resultado_carro["afectacion"]}px^2')
        else:
            data_main['mascara carro'].append('0px^2')
            data_main['area carro'].append('0px^2')
            data_main['afectacion'].append('0px^2')
    df= pd.DataFrame(data_main)
    return df
    
def procesar_deteccion_segundaria(carro, area_abolladura, area_rayon, instancia_img):
    area_carro, mascara_carro = buscar_area_por_instancia(carro, instancia_img)

    area_total_afectada = abs(area_carro - area_abolladura) if area_abolladura > 0 else 0
    area_total_afectada = abs(area_carro - area_rayon) if area_rayon > 0 else area_total_afectada
    if area_abolladura > 0 and area_rayon > 0:
        sum_area_principal = area_abolladura + area_rayon 
        area_total_afectada = sum_area_principal - area_carro

    return {'area carro': area_carro, 'mascara carro':mascara_carro ,'afectacion':area_total_afectada}
    
def procesar_dataframe(mascaras):
    if not mascaras:
        return 0

    formatted_mascaras= [f'mascara {id + 1}: {area}' for id, area in enumerate(mascaras)]
    return ", ".join(formatted_mascaras)

dataframe=procesar_deteccion_principal(results_abolladura, results_rayon, results_car)
estilo=dataframe.style.set_properties(**{'text-align':'rigth'})
estilo

,imagen,mascara abolladura,area abolladura,mascara rayon,area rayon,mascara carro,area carro,afectacion
0,1,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2
1,2,0px^2,0px^2,"mascara 1: 99, mascara 2: 230, mascara 3: 317, mascara 4: 87px^2",733px^2,0px^2,0px^2,733px^2
2,3,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2
3,4,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2
4,5,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2
5,6,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2
6,7,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2,0px^2
7,8,0px^2,0px^2,mascara 1: 527px^2,527px^2,0px^2,0px^2,527px^2
8,9,0px^2,0px^2,mascara 1: 303px^2,303px^2,mascara 1: 367066px^2,367066px^2,366763px^2
9,10,mascara 1: 66108px^2,66108px^2,0px^2,0px^2,0px^2,0px^2,66108px^2


In [7]:
# idea 2
# areas individuales dataframe
def calcular_area(mascara):
    sum_area = 0
    areas=[]
    if mascara is not None:
        for id, e in enumerate(mascara):
            x = e[:, 0]
            y = e[:, 1] 
            area = np.round(0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))).astype(int)
            #areas.append(area) #
            sum_area += area
        return sum_area, areas

def buscar_area_por_instancia(results, instancia_img):
    for i in results:
        if i.path == instancia_img:
            return calcular_area(i.masks.xy) if i.masks is not None else (0,[])
        
def procesar_deteccion_principal(abolladura, rayon, results_car):
    data_main = {'imagen':[], 'mascara abolladura':[], 'area abolladura':[], 'mascara rayon':[] ,'area rayon':[], 'mascara carro':[],'area carro':[], 'afectacion':[]}
    
    for id_imagen, e_abolladura in enumerate(abolladura):
        instancia_img = e_abolladura.path

        data_main['imagen'].append(id_imagen + 1)
        area_abolladura, mascara_abolladura = buscar_area_por_instancia(abolladura, instancia_img)
        data_main['mascara abolladura'].append(f'{mascara_abolladura}px^2')
        data_main['area abolladura'].append(f'{area_abolladura}px^2')
        
        area_rayon, mascara_rayon = buscar_area_por_instancia(rayon, instancia_img)
        data_main['mascara rayon'].append(f'{mascara_rayon}px^2')
        data_main['area rayon'].append(f'{area_rayon}px^2')
        
        if area_abolladura > 0 or area_rayon > 0:
            resultado_carro=procesar_deteccion_segundaria(results_car, area_abolladura, area_rayon, instancia_img)
            data_main['mascara carro'].append(f'{resultado_carro["mascara carro"]}px^2')
            data_main['area carro'].append(f'{resultado_carro["area carro"]}px^2')
            data_main['afectacion'].append(f'{resultado_carro["afectacion"]}px^2')
        else:
            data_main['mascara carro'].append('[]px^2')
            data_main['area carro'].append('0px^2')
            data_main['afectacion'].append('0px^2')
    df= pd.DataFrame(data_main)
    return df
    
def procesar_deteccion_segundaria(carro, area_abolladura, area_rayon, instancia_img):
    area_carro, mascara_carro = buscar_area_por_instancia(carro, instancia_img)

    area_total_afectada = abs(area_carro - area_abolladura) if area_abolladura > 0 else 0
    area_total_afectada = abs(area_carro - area_rayon) if area_rayon > 0 else area_total_afectada
    if area_abolladura > 0 and area_rayon > 0:
        sum_area_principal = area_abolladura + area_rayon 
        area_total_afectada = sum_area_principal - area_carro

    return {'area carro': area_carro, 'mascara carro':mascara_carro ,'afectacion':area_total_afectada}
    

dataframe=procesar_deteccion_principal(results_abolladura, results_rayon, results_car)
estilo=dataframe.style.set_properties(**{'text-align':'rigth'})
estilo

[99, 230, 317, 87]
[303]
[367066]
[66108]


,imagen,mascara abolladura,area abolladura,mascara rayon,area rayon,mascara carro,area carro,afectacion
0,1,[]px^2,0px^2,[]px^2,0px^2,[]px^2,0px^2,0px^2
1,2,[]px^2,0px^2,"[99, 230, 317, 87]px^2",733px^2,[]px^2,0px^2,733px^2
2,3,[]px^2,0px^2,[]px^2,0px^2,[]px^2,0px^2,0px^2
3,4,[]px^2,0px^2,[]px^2,0px^2,[]px^2,0px^2,0px^2
4,5,[]px^2,0px^2,[]px^2,0px^2,[]px^2,0px^2,0px^2
5,6,[]px^2,0px^2,[]px^2,0px^2,[]px^2,0px^2,0px^2
6,7,[]px^2,0px^2,[]px^2,0px^2,[]px^2,0px^2,0px^2
7,8,[]px^2,0px^2,[303]px^2,303px^2,[367066]px^2,367066px^2,366763px^2
8,9,[66108]px^2,66108px^2,[]px^2,0px^2,[]px^2,0px^2,66108px^2


In [208]:

import numpy as np
import pandas as pd

def calcular_area(mascara):
    areas = []
    if mascara is not None:
        for e in mascara:
            x = e[:, 0]
            y = e[:, 1] 
            area = np.round(0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))).astype(int)
            areas.append(area)
    return areas  # Si no hay máscaras, devuelve una lista vacía

def buscar_area_por_instancia(results, instancia_img):
    for i in results:
        if i.path == instancia_img:
            return calcular_area(i.masks.xy) if i.masks is not None else []  # Si no hay máscaras, devuelve una lista vacía

def procesar_deteccion_principal(abolladura, rayon):
    data_main = {'imagen': [], 'area': []}
    
    for id_imagen, e_abolladura in enumerate(abolladura):
        instancia_img = e_abolladura.path

        mascara_abolladura = buscar_area_por_instancia(e_abolladura, instancia_img)
        mascara_rayon = buscar_area_por_instancia(rayon, instancia_img)
        if not mascara_rayon:  # Si la lista de máscaras está vacía, añade una fila con máscara vacía
            data_main['imagen'].append(id_imagen + 1)
            data_main['area'].append(0)

        for area in mascara_rayon:
            data_main['imagen'].append(id_imagen + 1)
            data_main['area'].append(area)

    df = pd.DataFrame(data_main)
    return df

            
dataframe = procesar_deteccion_principal(results_abolladura, results_rayon)
estilo = dataframe.style.set_properties(**{'text-align': 'right'})
estilo


,imagen,mascara rayon
0,1,0
1,2,99
2,2,230
3,2,317
4,2,87
5,3,0
6,4,0
7,5,0
8,6,0
9,7,0


In [50]:
import numpy as np
import pandas as pd

def calcular_area(mascara):
    areas = []
    if mascara is not None:
        for e in mascara:
            x = e[:, 0]
            y = e[:, 1] 
            area = np.round(0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))).astype(int)
            areas.append(area)
    return areas  # Si no hay máscaras, devuelve una lista vacía

def buscar_area_por_instancia(results, instancia_img):
    for i in results:
        if i.path == instancia_img:
            return calcular_area(i.masks.xy) if i.masks is not None else []  # Si no hay máscaras, devuelve una lista vacía

def agregar_filas(data_main, id_imagen, etiqueta, mascara, columna):
    if not mascara: #not
        data_main['imagen'].append(id_imagen)
        data_main['etiqueta'].append(etiqueta)
        data_main[columna].append(0)
        
    elif isinstance(mascara, list):
        for area in mascara:
            data_main['imagen'].append(id_imagen)
            data_main['etiqueta'].append(etiqueta)
            data_main[columna].append(area)
    else:
        data_main['imagen'].append(id_imagen)
        data_main['etiqueta'].append(etiqueta)
        data_main[columna].append(mascara)

def procesar_deteccion_principal(abolladura, rayon):
    data_main = {'imagen': [], 'etiqueta':[],'area': []} #'mascara a': [],
    
    for id_imagen, e_abolladura in enumerate(abolladura):
        instancia_img = e_abolladura.path
        imagen_actual = id_imagen + 1
        
        mascara_abolladura = buscar_area_por_instancia(e_abolladura, instancia_img)
        agregar_filas(data_main, imagen_actual,'abolladura', mascara_abolladura, 'area')
        
        mascara_rayon = buscar_area_por_instancia(rayon, instancia_img)
        agregar_filas(data_main, imagen_actual,'rayon', mascara_rayon, 'area')

    df = pd.DataFrame(data_main)
    df['num_valor'] = df.groupby(['imagen', 'etiqueta']).cumcount()
    return df

dataframe = procesar_deteccion_principal(results_abolladura, results_rayon)
dataframe.groupby(["imagen", "etiqueta"], group_keys=True, as_index=True)[[ "num_valor", "area"]].apply(lambda x: x)

num_valor   area
imagen etiqueta                       
1      abolladura 0           0      0
       rayon      1           0      0
2      abolladura 2           0      0
       rayon      3           0     99
                  4           1    230
                  5           2    317
                  6           3     87
3      abolladura 7           0      0
       rayon      8           0      0
4      abolladura 9           0      0
       rayon      10          0      0
5      abolladura 11          0      0
       rayon      12          0      0
6      abolladura 13          0      0
       rayon      14          0      0
7      abolladura 15          0      0
       rayon      16          0      0
8      abolladura 17          0      0
       rayon      18          0    303
9      abolladura 19          0  66108
       rayon      20          0      0

In [50]:
for e in results_abolladura:
    print(e.masks)

None
None
None
None
None
None
None
None
ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
orig_shape: (675, 1013)
shape: torch.Size([1, 448, 640])
xy: [array([[     742.34,      288.43],
       [     740.76,      290.02],
       [     726.51,      290.02],
       [     724.93,       291.6],
       [     720.18,       291.6],
       [     717.01,      294.76],
       [     717.01,      296.35],
       [     715.43,      297.93],
       [     715.43,      299.51],
       [     713.85,       301.1],
       [     713.85,      302.68],
       [     685.36,      331.17],
       [     683.78,      331.17],
       [     683.78,      332.75],
       [     679.03,       337.5],
       [     677.44,       337.5],
       [    

# intento ganador

In [68]:
def calculate_area(segmented_masks): 
    total_area_of_masks= 0
    individual_mask_area= []
    if segmented_masks is not None:
        for vectors in segmented_masks: 
            coordinate_x= vectors[:, 0] # 
            coordinate_y= vectors[:, 1] 
            area_mask= np.round(0.5 * abs(np.dot(coordinate_x, np.roll(coordinate_y, 1)) - np.dot(coordinate_y, np.roll(coordinate_x, 1)))).astype(int) # Se basa en la formula Shoelace o lazada de Gauss. Utiliza la suma y la resta de los productos cruzados de las coordenadas de los vertices del poligono; el termino np.roll(y, 1) y np.roll(x, 1) crean versiones desplazadas circularmente, es esencial para que np.dot() calcule el producto punto de los vertices del poligono, la diferencia entre los productos cruzados se multiplicara por 1/2 tomando el valor absoluto.
            individual_mask_area.append(area_mask) # Permite guardar el area de cada mascara detectada por imagen segun la clase
            total_area_of_masks += area_mask # Permite conocer el valor total de las areas 
    return individual_mask_area, total_area_of_masks

def search_mask_by_image(segmentation_results, id_image):
    for attributes in segmentation_results:
        if attributes.path == id_image: # Confirmar que estamos buscando el resultado en la misma imagen por cada clase, con el atributo path obtenemos la direccion del archivo
            return calculate_area(i.masks.xy) if i.masks is not None else ([], 0) # Envia las coordenadas para el calculo del area si la mascara no esta vacia. De lo contrario, retornara como valor una lista vacia referente a las areas halladas y un equivalente a cero por el area total de la imagen.
    return None, None

def add_dataframe_rows(main_data, id_image, image_name, class_identification, mask_area_by_class, affectation):
    id_class = {'abolladura': 0, 'rayon': 1, 'carro': 2} # Asigna a cada clase un identificador de orden
    
    main_data['id_imagen'].append(id_image)
    main_data['imagen'].append(image_name)
    main_data['ClaseMascara'].append(class_identification)
    main_data['area x mascara'].append(0)
    main_data['afectacion x imagen'].append(affectation)
    """if not mask_area_by_class:
        main_data['id_imagen'].append(id_image)
        main_data['imagen'].append(image_name)
        main_data['ClaseMascara'].append(class_identification)
        main_data['area x mascara'].append(0)
        main_data['afectacion x imagen'].append(affectation)"""
    if  isinstance(mask_area_by_class(list)): #Verifica si las areas de cada mascara segmentada esta en una lista
        for area in mask_area_by_class: # recorre cada area de la lista de areas, permite asignar por cada area una fila en el dataframe.
            main_data['id_imagen'].append(id_image)
            main_data['imagen'].append(image_name)
            main_data['ClaseMascara'].append(class_identification)
            main_data['area x mascara'].append(area)
            main_data['afectacion x imagen'].append(affectation)

    main_data['id_mascara'] = [id_class[class_name] for class_name in main_data['ClaseMascara']] # Se añade una nueva columna que contendra la asignacion de cada clase 

def process_segmentation_results(abolladura, rayon, car):
    main_data = {'id_imagen': [], 'imagen':[],'ClaseMascara': [], 'area x mascara': [], 'afectacion x imagen':[]} # Creacion de la estructura inicial para llenar el dataframe, las columnas tienen asignado listas vacias.

    for image_number, attributes in enumerate(abolladura): # Recorre los resultados de la segmentacion abolladura, una instancia obtendra el identificador de recorrido.
        img_directory = attributes.path # Se obtiene el atributo de directorio en imagen
        id_image = image_number + 1 # Incrementa el contador para asignarle una identificación única a la imagen
        
        areas_abolladura, total_abolladura_area = search_mask_by_image(attributes, img_directory) 
        areas_rayon, total_rayon_area = search_mask_by_image(rayon, img_directory)
        
        if total_abolladura_area is not None and total_abolladura_area > 0 or total_rayon_area is not None and total_rayon_area > 0:
            areas_car, total_car_area = search_mask_by_image(car, img_directory)
            afectacion_imagen= calculating_car_affectation(total_car_area, total_abolladura_area, total_rayon_area)
            
            add_dataframe_rows(main_data, id_image, img_directory, 'abolladura', areas_abolladura, afectacion_imagen)
            add_dataframe_rows(main_data, id_image, img_directory, 'rayon', areas_rayon, afectacion_imagen)
            add_dataframe_rows(main_data, id_image, img_directory, 'carro', areas_car, afectacion_imagen)
        else:
            add_dataframe_rows(main_data, id_image, img_directory,'abolladura', areas_abolladura, 0)
            add_dataframe_rows(main_data, id_image, img_directory,'rayon', areas_rayon, 0)
            add_dataframe_rows(main_data, id_image, img_directory, 'carro', [], 0)

    df = pd.DataFrame(main_data) # Convierte la lista de diccionarios a dataframe
    df['nombre'] = df['imagen'].apply(lambda ruta: os.path.basename(ruta)) #  Obtener solo el nombre del archivo
    grouped_columns=df.groupby(['id_imagen', 'nombre', 'afectacion x imagen','id_mascara','ClaseMascara'], group_keys=True, as_index=True)[[ 'area x mascara']].apply(lambda x : x) # Se utiliza una agrupacion serial, se agrupa por imagen la afectacion total y los tres tipos de clase, independiente quedan las areas que pertenecen a cada clase.
    return grouped_columns

def calculating_car_affectation(total_car_area, total_abolladura_area, total_rayon_area): 
    total_area_affected  = abs(total_car_area - total_abolladura_area) if total_abolladura_area is not None and total_abolladura_area > 0 else abs(total_car_area - total_rayon_area) if total_rayon_area is not None and total_rayon_area > 0 else 0 # verifica si existe un valor valido de abolladura para encontrar el valor de area afectada en el carro, de lo contrario verifica lo mismo para rayon, asigna como valor de afectacion 0 en caso de no ser valido.
    
    if total_abolladura_area is not None and total_rayon_area is not None and total_abolladura_area > 0 and total_rayon_area > 0:
        total_area_affected = total_abolladura_area + total_rayon_area - total_car_area # Si ambos tipos de afectacion existen en una imagen, se sumaran primero y restaran con el area del carro.
    return total_area_affected

dataframe = process_segmentation_results(results_abolladura, results_rayon, results_car)
print(dataframe)

                                                                                                             area x mascara
id_imagen nombre                                             afectacion x imagen id_mascara ClaseMascara                   
1         20170531_010133.jpg                                0                   0          abolladura   0                0
                                                                                 1          rayon        1                0
                                                                                 2          carro        2                0
2         OIP (1).jpg                                        733                 0          abolladura   3                0
                                                                                 1          rayon        4               99
                                                                                                         5              230
        

In [41]:
# funciones de pandas
def calculate_area(segmented_masks): 
    total_area_of_masks= 0
    individual_mask_area= []
    if segmented_masks is not None:
        for vectors in segmented_masks: 
            coordinate_x= vectors[:, 0] # 
            coordinate_y= vectors[:, 1] 
            area_mask= np.round(0.5 * abs(np.dot(coordinate_x, np.roll(coordinate_y, 1)) - np.dot(coordinate_y, np.roll(coordinate_x, 1)))).astype(int) # Se basa en la formula Shoelace o lazada de Gauss. Utiliza la suma y la resta de los productos cruzados de las coordenadas de los vertices del poligono; el termino np.roll(y, 1) y np.roll(x, 1) crean versiones desplazadas circularmente, es esencial para que np.dot() calcule el producto punto de los vertices del poligono, la diferencia entre los productos cruzados se multiplicara por 1/2 tomando el valor absoluto.
            individual_mask_area.append(area_mask) # Permite guardar el area de cada mascara detectada por imagen segun la clase
            total_area_of_masks += area_mask # Permite conocer el valor total de las areas 
    return individual_mask_area, total_area_of_masks

def search_mask_by_image(segmentation_results, id_image):
    for attributes in segmentation_results:
        if attributes.path == id_image: # Confirmar que estamos buscando el resultado en la misma imagen por cada clase, con el atributo path obtenemos la direccion del archivo
            return calculate_area(i.masks.xy) if i.masks is not None else ([], 0)
    return None, None
        
def add_dataframe_rows(main_data, image_name, class_identification, mask_area_by_class, affectation): # Se encarga de llenar la lista vacia por imagen del diccionario
    main_data['imagen'].append(image_name)
    main_data['ClaseMascara'].append(class_identification)
    main_data['afectacion x imagen'].append(affectation)
    main_data['area x mascara'].append(mask_area_by_class)
    
    """if not mask_area_by_class:
        main_data['area x mascara'].append(mask_area_by_class) #
    else:
        main_data['area x mascara'].append(mask_area_by_class)""" #

def process_segmentation_results(abolladura, rayon, car):
    main_data = {'imagen':[],'ClaseMascara': [], 'area x mascara': [], 'afectacion x imagen':[]} # Creacion de la estructura inicial para llenar el dataframe, las columnas tienen asignado listas vacias.
    
    for attributes in abolladura: # Recorre los resultados de la segmentacion abolladura,
        img_directory = attributes.path # Se obtiene el atributo de directorio en imagen
                
        areas_abolladura, total_abolladura_area = search_mask_by_image(attributes, img_directory)
        areas_rayon, total_rayon_area = search_mask_by_image(rayon, img_directory)
        
        if total_abolladura_area is not None and total_abolladura_area > 0 or total_rayon_area is not None and total_rayon_area > 0:
            areas_car, total_car_area = search_mask_by_image(car, img_directory)
            afectacion_imagen= calculating_car_affectation(total_car_area, total_abolladura_area, total_rayon_area)
            
            add_dataframe_rows(main_data, img_directory, 'abolladura', areas_abolladura, afectacion_imagen)
            add_dataframe_rows(main_data, img_directory, 'rayon', areas_rayon, afectacion_imagen)
            add_dataframe_rows(main_data, img_directory, 'carro', areas_car, afectacion_imagen)
        else:
            add_dataframe_rows(main_data, img_directory, 'abolladura', areas_abolladura, 0)
            add_dataframe_rows(main_data, img_directory, 'rayon', areas_rayon, 0)
            add_dataframe_rows(main_data, img_directory, 'carro', [], 0)

    df = pd.DataFrame(main_data) # Convierte la lista de diccionarios a dataframe
    df['nombre'] = df['imagen'].apply(lambda ruta: os.path.basename(ruta)) #  Obtener solo el nombre del archivo
    encoder_class= LabelEncoder() 
    df['id_mascara']= encoder_class.fit_transform(df.ClaseMascara) # transforma los datos categoricos en numericos, asignandolos como identificados a una nueva columna.
    explode= df.explode('area x mascara') # Realiza la explotacion de las listas dentro del DataFrame, duplicando las filas relacionadas hasta que cada valor de la columna area x mascara tenga su propia posicion
    grouped_columns=explode.groupby(['nombre', 'afectacion x imagen','id_mascara', 'ClaseMascara'], group_keys=True, as_index=True)[['area x mascara']].apply(lambda x : x) # Se utiliza una agrupacion serial, se agrupa por imagen la afectacion total y los tres tipos de clase, independiente quedan las areas que pertenecen a cada clase.
    return grouped_columns

def calculating_car_affectation(total_car_area, total_abolladura_area, total_rayon_area): 
    total_area_affected  = abs(total_car_area - total_abolladura_area) if total_abolladura_area is not None and total_abolladura_area > 0 else abs(total_car_area - total_rayon_area) if total_rayon_area is not None and total_rayon_area > 0 else 0 # verifica si existe un valor valido de abolladura para encontrar el valor de area afectada en el carro, de lo contrario verifica lo mismo para rayon, asigna como valor de afectacion 0 en caso de no ser valido.
    
    if total_abolladura_area is not None and total_rayon_area is not None and total_abolladura_area > 0 and total_rayon_area > 0:
        total_area_affected = total_abolladura_area + total_rayon_area - total_car_area # Si ambos tipos de afectacion existen en una imagen, se sumaran primero y restaran con el area del carro.
    return total_area_affected

dataframe = process_segmentation_results(results_abolladura, results_rayon, results_car)
print(dataframe)

                                                                                                  area x mascara
nombre                                             afectacion x imagen id_mascara ClaseMascara                  
20170531_010133.jpg                                0                   0          abolladura   0               0
                                                                       1          carro        2               0
                                                                       2          rayon        1               0
OIP (1).jpg                                        733                 0          abolladura   3               0
                                                                       1          carro        5               0
                                                                       2          rayon        4              99
                                                                                               4

In [44]:
# funciones de pandas
def calcular_area(mascara):
    sum_areas = 0
    areas = []
    if mascara is not None:
        for e in mascara:
            x = e[:, 0]
            y = e[:, 1] 
            area = np.round(0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))).astype(int)
            areas.append(area)
            sum_areas += area
    return areas, sum_areas 

def buscar_area_por_instancia(results, instancia_img):
    for i in results:
        if i.path == instancia_img:
            return calcular_area(i.masks.xy) if i.masks is not None else ([], 0) 
    return [], 0
        
def agregar_filas(data_main, nombre_imagen, etiqueta, mascara, afectacion_imagen):
    data_main['imagen'].append(nombre_imagen)
    data_main['ClaseMascara'].append(etiqueta)
    data_main['afectacion x imagen'].append(afectacion_imagen)
    if not mascara:
        data_main['area x mascara'].append(0)
    else:
        data_main['area x mascara'].append(mascara)

def procesar_deteccion_principal(abolladura, rayon, carro):
    data_main = {'imagen':[],'ClaseMascara': [], 'area x mascara': [], 'afectacion x imagen':[]}

    for id_imagen, e_abolladura in enumerate(abolladura):
        instancia_img = e_abolladura.path
        imagen_actual = id_imagen + 1
        
        mascara_abolladura, area_abolladura = buscar_area_por_instancia(e_abolladura, instancia_img)
        mascara_rayon, area_rayon = buscar_area_por_instancia(rayon, instancia_img)
        
        if area_abolladura is not None and area_abolladura > 0 or area_rayon is not None and area_rayon > 0:
            mascara_carro, area_total_carro = buscar_area_por_instancia(carro, instancia_img)
            afectacion_imagen= procesar_areas(area_total_carro, area_abolladura, area_rayon)
            
            agregar_filas(data_main, instancia_img, 'abolladura', mascara_abolladura, afectacion_imagen)
            agregar_filas(data_main, instancia_img, 'rayon', mascara_rayon, afectacion_imagen)
            agregar_filas(data_main, instancia_img, 'carro', mascara_carro, afectacion_imagen)

        else:
            agregar_filas(data_main, instancia_img,'abolladura', mascara_abolladura, 0)
            agregar_filas(data_main, instancia_img,'rayon', mascara_rayon, 0)
            agregar_filas(data_main, instancia_img, 'carro', [], 0)

    df = pd.DataFrame(data_main)
    df['nombre'] = df['imagen'].apply(lambda ruta: os.path.basename(ruta))
    encoder_class= LabelEncoder()
    df['id_mascara']= encoder_class.fit_transform(df.ClaseMascara)
    explode= df.explode('area x mascara')
    group=explode.groupby(['nombre', 'afectacion x imagen','ClaseMascara'], group_keys=True, as_index=True)[['area x mascara']].apply(lambda x : x)
    return group

def procesar_areas(area_carro, area_abolladura, area_rayon): 
    
    area_total_afectada = abs(area_carro - area_abolladura) if area_abolladura is not None and area_abolladura > 0 else abs(area_carro - area_rayon) if area_rayon is not None and area_rayon > 0 else 0
    
    if area_abolladura is not None and area_rayon is not None and area_abolladura > 0 and area_rayon > 0:
        area_total_afectada = area_abolladura + area_rayon - area_carro
    return area_total_afectada

dataframe = procesar_deteccion_principal(results_abolladura, results_rayon, results_car)
print(dataframe)

                                                                                       area x mascara
nombre                                             afectacion x imagen ClaseMascara                  
20170531_010133.jpg                                0                   abolladura   0               0
                                                                       carro        2               0
                                                                       rayon        1               0
OIP (1).jpg                                        733                 abolladura   3               0
                                                                       carro        5               0
                                                                       rayon        4              99
                                                                                    4             230
                                                                                  

In [54]:
# Intento reset, eliminacion jerarquia
def calcular_area(mascara):
    sum_areas = 0
    areas = []
    if mascara is not None:
        for e in mascara:
            x = e[:, 0]
            y = e[:, 1] 
            area = np.round(0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))).astype(int)
            areas.append(area)
            sum_areas += area
    return areas, sum_areas  

def buscar_area_por_instancia(results, instancia_img):
    for i in results:
        if i.path == instancia_img:
            return calcular_area(i.masks.xy) if i.masks is not None else ([], 0)
    return 0, 0

def agregar_filas(data_main, id_imagen, etiqueta, mascara, area_mascara, afectacion_total):
    if mascara is None or (isinstance(mascara, list) and not mascara):
        mascara = 0
    if isinstance(mascara, list):
        for area in mascara:
            data_main.append({'imagen': id_imagen, 'tipo afectacion': etiqueta, 'area x mascara': area, 'afectacion x mascara': area_mascara, 'afectacion x imagen': afectacion_total})
    else:
        data_main.append({'imagen': id_imagen, 'tipo afectacion': etiqueta, 'area x mascara': mascara, 'afectacion x mascara': area_mascara, 'afectacion x imagen': afectacion_total})

def procesar_detecciones(abolladura, rayon, carro):
    data_main = [] 
    for id_imagen, e_abolladura in enumerate(abolladura):
        instancia_img = e_abolladura.path
        imagen_actual = id_imagen + 1
        
        mascara_abolladura, area_total_abolladura = buscar_area_por_instancia(e_abolladura, instancia_img)
        mascara_rayon, area_total_rayon = buscar_area_por_instancia(rayon, instancia_img)

        if area_total_abolladura is not None and area_total_abolladura > 0 or area_total_rayon is not None and area_total_rayon > 0:
            mascara_carro, area_total_carro = buscar_area_por_instancia(carro, instancia_img)
            afectacion_total= procesar_areas(area_total_carro, area_total_abolladura, area_total_rayon)
            agregar_filas(data_main, imagen_actual, 'abolladura', mascara_abolladura, area_total_abolladura, afectacion_total['area afectada imagen'])
            agregar_filas(data_main, imagen_actual, 'rayon', mascara_rayon, area_total_rayon, afectacion_total['area afectada imagen'])
            agregar_filas(data_main, imagen_actual, 'carro', mascara_carro, area_total_carro, afectacion_total['area afectada imagen'])
        else:
            agregar_filas(data_main, imagen_actual, 'abolladura', mascara_abolladura, 0, 0)
            agregar_filas(data_main, imagen_actual, 'rayon', mascara_rayon, 0, 0)
            agregar_filas(data_main, imagen_actual, 'carro', 0, 0, 0)

    df = pd.DataFrame(data_main)
    df['id_mascara'] = df.groupby(['imagen', 'tipo afectacion']).cumcount() 
    group= df.groupby(["imagen", "afectacion x imagen","tipo afectacion", 'afectacion x mascara', 'id_mascara'], as_index=True, group_keys=True).apply(lambda x: x[['area x mascara']]).reset_index()
    return group

def procesar_areas(area_carro, area_abolladura, area_rayon): 
    
    area_total_afectada = abs(area_carro - area_abolladura) if area_abolladura is not None and area_abolladura > 0 else 0
    area_total_afectada= abs(area_carro - area_rayon) if area_rayon is not None and area_rayon > 0 else 0
    if area_abolladura is not None and area_abolladura > 0 and area_rayon is not None and area_rayon > 0:
        sum_area_principal = area_abolladura + area_rayon 
        area_total_afectada = sum_area_principal - area_carro
    else:
        area_total_afectada= area_total_afectada

    return {'area abolladura': area_total_afectada, 'area rayon': area_total_afectada, 'area afectada imagen': area_total_afectada}

dataframe = procesar_detecciones(results_abolladura, results_rayon, results_car)
dataframe

,imagen,afectacion x imagen,tipo afectacion,afectacion x mascara,id_mascara,level_5,area x mascara
0,1,0,abolladura,0,0,0,0
1,1,0,carro,0,0,2,0
2,1,0,rayon,0,0,1,0
3,2,733,abolladura,0,0,3,0
4,2,733,carro,0,0,8,0
5,2,733,rayon,733,0,4,99
6,2,733,rayon,733,1,5,230
7,2,733,rayon,733,2,6,317
8,2,733,rayon,733,3,7,87
9,3,0,abolladura,0,0,9,0


In [197]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                             'Parrot', 'Parrot'],
                   'Max Speed': [[380., 370.], 3 , 24., 26.]})

e=df.explode('Max Speed')
e.groupby("Animal", group_keys=True)[['Max Speed']].apply(lambda x: x)


Max Speed
Animal            
Falcon 0     380.0
       0     370.0
       1         3
Parrot 2      24.0
       3      26.0

In [41]:
# Función de área en formato pixel actualizada

def calcular_area(vertices):
    coordenada_x = vertices[:, 0] # Asigna todas filas correspondientes a la primer columna como coordenada X del formato masks que entrega por medio de arrays los vertices (x , y) de los puntos de contorno por medio de una lista para cada región segmentada.
    coordenada_y = vertices[:, 1] # Asigna todas filas correspondientes a la segunda columna como coordenada Y del formato masks que entrega por medio de arrays los componentes (x , y) del vertice referentes a los puntos de contorno por medio de una lista para cada región segmentada.
    return 0.5 * abs(np.dot(coordenada_x, np.roll(coordenada_y, 1)) - np.dot(coordenada_y, np.roll(coordenada_x, 1))) # Se basa en la formula Shoelace o lazada de Gauss. Utiliza la suma y la resta de los productos cruzados de las coordenadas de los vertices del poligono; el termino np.roll(y, 1) y np.roll(x, 1) crean versiones desplazadas circularmente, es esencial para que np.dot() calcule el producto punto de los vertices del poligono, la diferencia entre los productos cruzados se multiplicara por 1/2 tomando el valor absoluto.

def procesar_mascaras(mascaras, mensaje):
    area_total_objeto = 0.0
    print(f'{mensaje}: ')
    for id_mascara, coordenadas in enumerate(mascaras): # Recorre con una instancia los valores de las coordenadas que obtuvo de las mascaras individualmente.
        coordenada_mascara = calcular_area(coordenadas)
        area_total_objeto += coordenada_mascara # Al obtener el area individual de cada mascara en un objeto ira sumando en caso de encontrar mas mascaras en una instancia.
        print(f'Mascara {id_mascara + 1}, area: {coordenada_mascara} px^2 ') 
    return area_total_objeto

def procesar_imagen(segmentacion_base, segmentacion_car):
    for id_objeto, mascara_central in enumerate(segmentacion_base, 0): #Recorre los resultados que contienen la información de las segmentaciones de la mascara principal, obteniendo su posición y valor.
        
        print(f'\nImagen {id_objeto + 1}: {mascara_central.path}') # Imprimira la posición del objeto, ademas de la ubicación del archivo en el equipo.
        
        mascaras_principales = mascara_central.masks.xy if mascara_central.masks is not None else None # Guarda en una variable los atributos que encontro sobre las mascaras de cada objeto, se compone de data, orig_shape, formatos de valores para las coordenadas; xy (formato en valores pixel) y xyn (formato en valores normalizados al tamaño de la imagen).
        if mascaras_principales is not None and len(mascaras_principales) > 0: # Permite verificar si existen mascaras de segmentación en el objeto, debe contener la variable un valor mayor a 0.
            area_base_total = procesar_mascaras(mascaras_principales, "MASCARAS RAYON ABOLLADURA")
            
            for mascara_auxiliar in segmentacion_car: # Recorre con una instancia los resultados de los atributos de segmentación carro.
                if mascara_central.path == mascara_auxiliar.path: # Verifica si la instancia en la que se esta recorriendo los resultados de las mascaras de carro sea el mismo objeto en el que se hallo la mascara principal.
                    mascaras_carro = mascara_auxiliar.masks.xy if mascara_auxiliar.masks is not None else None # Almacenara las instancias de las coordenadas en formato pixel de las mascaras segmentadas en una imagen, si no encuentra almacenara el valor de None. Es recomendable utilizar el mismo formato con el que solicito las coordenadas de la mascara principal.
                    if mascaras_carro is not None and len(mascaras_carro) > 0: # Verifica que exista un valor valido en la variable que contiene las mascaras de segmentación.
                        area_carro_total = procesar_mascaras(mascaras_carro, "MASCARAS CARRO")
                        area_total_afectada = area_carro_total - area_base_total 
                        print(f'Área total afectada en el carro: {area_total_afectada} px^2')
                    else:
                        area_total_afectada = area_base_total # Al no contener mascaras de segmentación, el area total afectada pasa a ser solo el area total de la mascara principal.
                        print(f'Área total afectada en el carro: {area_total_afectada} px^2')
        else:
            print(f'No se encontraron segmentaciones en la imagen! ') # Imprime por pantalla cuando el valor de la variable que contiene las mascaras de segmentación principal es None.
                
#procesar_imagen(mascara_principal, mascara_car)

print('\nProceso terminado! ')


Imagen 1: C:\Users\matrix\pruebayolo\pruebaimagenes\20170531_010133.jpg
No se encontraron segmentaciones en la imagen! 

Imagen 2: C:\Users\matrix\pruebayolo\pruebaimagenes\OIP (1).jpg
MASCARAS RAYON ABOLLADURA: 
Mascara 1, area: 98.75 px 
Mascara 2, area: 230.25 px 
Mascara 3, area: 317.0 px 
Mascara 4, area: 87.4921875 px 
Área total afectada en el carro: 733.4921875 px^2

Imagen 3: C:\Users\matrix\pruebayolo\pruebaimagenes\R.jpg
No se encontraron segmentaciones en la imagen! 

Imagen 4: C:\Users\matrix\pruebayolo\pruebaimagenes\abolladura-en-un-coche-632c58a6e2ea5.jpg
No se encontraron segmentaciones en la imagen! 

Imagen 5: C:\Users\matrix\pruebayolo\pruebaimagenes\abolladuraautos-fd03b3bae67d7363172c41cc00198c51-1200x800.jpg
No se encontraron segmentaciones en la imagen! 

Imagen 6: C:\Users\matrix\pruebayolo\pruebaimagenes\big_ray.jpg
No se encontraron segmentaciones en la imagen! 

Imagen 7: C:\Users\matrix\pruebayolo\pruebaimagenes\coche-después-de-una-ruina-la-necesidad-ser-

In [28]:
#Intento eliminacion de cabecera superpuesta

import pandas as pd

# Crear un DataFrame de ejemplo
data = {'Grupo': ['A', 'A', 'B', 'B', 'C', 'C'],
        'Valor1': [10, 20, 30, 40, 50, 60],
        'Valor2': [100, 200, 300, 400, 500, 600]}
df = pd.DataFrame(data)

df.explode(column=['Valor1', 'Valor2'])
grouped_df = df.groupby('Grupo', group_keys=True)[['Valor1', 'Valor2']].apply(lambda x: x)
print(grouped_df.to_string(header=False))



Grupo           
A     0  10  100
      1  20  200
B     2  30  300
      3  40  400
C     4  50  500
      5  60  600


In [26]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'Nombre': ['Alice', 'Bob', 'Charlie'],
        'Edad': [25, 30, 35],
        'Puntuación': [85, 90, 88]}
df = pd.DataFrame(data)

# Mostrar el DataFrame sin cabeceras
print(df.to_string(header=False))


0    Alice  25  85
1      Bob  30  90
2  Charlie  35  88


In [27]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'Grupo': ['A', 'A', 'B', 'B', 'C', 'C'],
        'Valor1': [10, 20, 30, 40, 50, 60],
        'Valor2': [100, 200, 300, 400, 500, 600]}
df = pd.DataFrame(data)

# Agrupar por la columna 'Grupo'
grouped_df = df.groupby('Grupo').sum()

# Ocultar las cabeceras al mostrar el DataFrame
print(grouped_df.to_string(header=False))


Grupo           
A       30   300
B       70   700
C      110  1100
